In [ ]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6YaKqIfpOnkAwSemb85j")
project = rf.workspace("testing-3j1sj").project("data-prep-lsyc8")
version = project.version(2)
dataset = version.download("yolov11")

In [ ]:
%pip install ultralytics
from ultralytics import YOLO
from pathlib import Path
import pandas as pd
import torch
import os
import shutil

In [ ]:
# Adjusting the folder for yolov11. However valid folder needs to be change into val by hand.

# Define the base directory
base_dir = 'data-prep-2' # Change as needed
# Step 1: Create new directories for consolidated images and labels
new_images_dir = os.path.join(base_dir, 'images')
new_labels_dir = os.path.join(base_dir, 'labels')
os.makedirs(new_images_dir, exist_ok=True)
os.makedirs(new_labels_dir, exist_ok=True)
# Define the splits (subfolders)
splits = ['train', 'test', 'valid']
# Step 2: Copy files from each split into the new structure
for split in splits:
    # Define the source directories for images and labels
    src_images_dir = os.path.join(base_dir, split, 'images')
    src_labels_dir = os.path.join(base_dir, split, 'labels')
    # Define the destination directories (creating subdirectories named after each split)
    dest_images_dir = os.path.join(new_images_dir, split)
    dest_labels_dir = os.path.join(new_labels_dir, split)
    os.makedirs(dest_images_dir, exist_ok=True)
    os.makedirs(dest_labels_dir, exist_ok=True)
    # Copy each image file to the corresponding new images subfolder
    for file_name in os.listdir(src_images_dir):
        src_file = os.path.join(src_images_dir, file_name)
        dest_file = os.path.join(dest_images_dir, file_name)
        shutil.copy(src_file, dest_file)
    # Copy each label file to the corresponding new labels subfolder
    for file_name in os.listdir(src_labels_dir):
        src_file = os.path.join(src_labels_dir, file_name)
        dest_file = os.path.join(dest_labels_dir, file_name)
        shutil.copy(src_file, dest_file)

print("Files copied to new structure successfully.")
# Step 3: Delete the old train, test, and valid folders
for split in splits:
    folder_path = os.path.join(base_dir, split)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    else:
        print(f"Folder does not exist: {folder_path}")

print("Old folders deleted.")


In [ ]:
torch.cuda.empty_cache() # Free the GPU memory just in case of a problem
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True # This one also kind of solves memory issues if there are any but not too reliable.
model = YOLO('yolo11m.pt') # Selecting a model. 
''' All these needs to be trained for around 10 to 15 epochs on the final dataset
yolo11n.pt -> Nano
yolo11s.pt -> Small
yolo11m.pt -> Medium
yolo11l.pt -> Large
yolo11x.pt -> XL
'''

results = model.train(
    data='/content/data.yaml',  # Path to your data config. Needs adjustment depending on where this is running.
    epochs=15,                  # Number of training epochs
    imgsz=640,                  # By defaul all the data is 640x640. So this can be left alone.
    batch=16,                   # Batch size
    device='cuda'               # Set to 0 for GPU, 'cpu' for CPU
)
''' 
For the moment rest of the adjustments can be left alone or multiple combinations of various adjustments can be tested in future. 
Refer to fastai: TODO: Find wtf was that function that combined different hyperparameters. and tested all of them out which then gave you the resul combination.
'''

In [ ]:
'''
This part is purely for COLAB, there is no reason to run this on local.
'''
import locale
print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
#print(locale.getpreferredencoding)
!zip -r /content/runs/detect/train7 /content/runs/detect/train7
from google.colab import files  # type: ignore
files.download("/content/runs/detect/train7.zip")